In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os


for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Sentiment Analysis on IMDB data using SVM

### Dataset Exploration and preprocessing

In [ ]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import seaborn as sns
import  dask

In [ ]:
ps = PorterStemmer()

In [ ]:
df = pd.read_csv("../input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv")
df.head()

In [ ]:
stopwords_eng = stopwords.words('english')
stopwords_eng.append('br')
#stopwords_eng

In [ ]:
sent = nltk.sent_tokenize(df.iloc[4]['review'])
sent

In [ ]:
corpus = []
review = []
for i in sent:
    review.extend(nltk.word_tokenize(re.sub('[^a-zA-Z]', ' ', i).lower()))
    #ps.stem(word)
    review = [ps.stem(word) for word in review if word not in stopwords_eng]
    
print(review)

In [ ]:
corpus=[]
for i in range(5000):
    #review = re.sub('[^a-zA-Z]',' ',df['review'][i]) # chalega
    #review = review.lower()
    #review = review.split()
    #review = [ps.stem(word) for word in review if word not in set(stopwords.words('english'))]
    #review=' '.join(review)
    review = nltk.word_tokenize(re.sub('[^a-zA-Z]', ' ', df['review'][i]).lower())
    #ps.stem(word)
    review = [ps.stem(word) for word in review if word not in stopwords_eng]
    review=' '.join(review)
    corpus.append(review)
#corpus

In [ ]:
import pandas as pd
prep_data = pd.read_csv("../input/imdbmoviereviewpreprocessedstemming50k/prepd_data.csv")
prep_data.head()

In [ ]:
corpus = list(prep_data['review'])
corpus[0]

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000)
X = cv.fit_transform(corpus).toarray()

# rows of X will be corresponding sentences and columns will be corresponding to word
y = prep_data['sentiment_label']

In [ ]:
print(X[0].sum())
print(X.shape)

In [ ]:
#corpus has 68992 unique words , how many of them you want to consider(depending upon frequency)?????
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.5, shuffle=True)

In [ ]:
test_corpus=[]
for i in range(45000,50000):    
    review1 = nltk.word_tokenize(re.sub('[^a-zA-Z]', ' ', df['review'][i]).lower())
    #ps.stem(word)--
    review1 = [ps.stem(word) for word in review1 if word not in stopwords_eng]
    review1 =' '.join(review1)
    test_corpus.append(review1)

In [ ]:
X2 = cv.transform(test_corpus).toarray()

# rows of X will be corresponding sentences and columns will be corresponding to word
    
y2 = pd.get_dummies(df['sentiment'])
y2 = y2.iloc[45000:50000,1].values
X2.shape

In [ ]:
y_pred2 = model1.predict(X2)
cm2 = confusion_matrix(y2,y_pred2)
accuracy2= accuracy_score(y2,y_pred2)

In [ ]:
accuracy2

## **TF-IDF**

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
cv = TfidfVectorizer(max_features=5000, ngram_range=(1, 3))
X = cv.fit_transform(list(prep_data['review'])).toarray()
y = prep_data['sentiment_label']

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.5, shuffle=True)

In [ ]:
from sklearn.linear_model import SGDClassifier 
model2 = SGDClassifier().fit(x_train,y_train,sample_weight=None)

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
y_pred_svm = model2.predict(x_test)
cm = confusion_matrix(y_test,y_pred_svm)
accuracy= accuracy_score(y_test,y_pred_svm)

In [ ]:
cm

In [ ]:
accuracy

### Linear SVM from Scratch ###

In [ ]:
"""
Multiclass SVMs (Crammer-Singer formulation).
A pure Python re-implementation of:
Large-scale Multiclass Support Vector Machine Training via Euclidean Projection onto the Simplex.
Mathieu Blondel, Akinori Fujino, and Naonori Ueda.
ICPR 2014.
http://www.mblondel.org/publications/mblondel-icpr2014.pdf
"""

import numpy as np

from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils import check_random_state
from sklearn.preprocessing import LabelEncoder


def projection_simplex(v, z=1):
    """
    Projection onto the simplex:
        w^* = argmin_w 0.5 ||w-v||^2 s.t. \sum_i w_i = z, w_i >= 0
    """
    # For other algorithms computing the same projection, see
    # https://gist.github.com/mblondel/6f3b7aaad90606b98f71
    n_features = v.shape[0]
    u = np.sort(v)[::-1]
    cssv = np.cumsum(u) - z
    ind = np.arange(n_features) + 1
    cond = u - cssv / ind > 0
    rho = ind[cond][-1]
    theta = cssv[cond][-1] / float(rho)
    w = np.maximum(v - theta, 0)
    return w


class MulticlassSVM(BaseEstimator, ClassifierMixin):

    def __init__(self, C=1, max_iter=50, tol=0.05,
                 random_state=None, verbose=0):
        self.C = C
        self.max_iter = max_iter
        self.tol = tol,
        self.random_state = random_state
        self.verbose = verbose

    def _partial_gradient(self, X, y, i):
        # Partial gradient for the ith sample.
        g = np.dot(X[i], self.coef_.T) + 1
        g[y[i]] -= 1
        return g

    def _violation(self, g, y, i):
        # Optimality violation for the ith sample.
        smallest = np.inf
        for k in range(g.shape[0]):
            if k == y[i] and self.dual_coef_[k, i] >= self.C:
                continue
            elif k != y[i] and self.dual_coef_[k, i] >= 0:
                continue

            smallest = min(smallest, g[k])

        return g.max() - smallest

    def _solve_subproblem(self, g, y, norms, i):
        # Prepare inputs to the projection.
        Ci = np.zeros(g.shape[0])
        Ci[y[i]] = self.C
        beta_hat = norms[i] * (Ci - self.dual_coef_[:, i]) + g / norms[i]
        z = self.C * norms[i]

        # Compute projection onto the simplex.
        beta = projection_simplex(beta_hat, z)

        return Ci - self.dual_coef_[:, i] - beta / norms[i]

    def fit(self, X, y):
        n_samples, n_features = X.shape

        # Normalize labels.
        self._label_encoder = LabelEncoder()
        y = self._label_encoder.fit_transform(y)

        # Initialize primal and dual coefficients.
        n_classes = len(self._label_encoder.classes_)
        self.dual_coef_ = np.zeros((n_classes, n_samples), dtype=np.float64)
        self.coef_ = np.zeros((n_classes, n_features))

        # Pre-compute norms.
        norms = np.sqrt(np.sum(X ** 2, axis=1))

        # Shuffle sample indices.
        rs = check_random_state(self.random_state)
        ind = np.arange(n_samples)
        rs.shuffle(ind)

        violation_init = None
        for it in range(self.max_iter):
            violation_sum = 0

            for ii in range(n_samples):
                i = ind[ii]

                # All-zero samples can be safely ignored.
                if norms[i] == 0:
                    continue

                g = self._partial_gradient(X, y, i)
                v = self._violation(g, y, i)
                violation_sum += v

                if v < 1e-12:
                    continue

                # Solve subproblem for the ith sample.
                delta = self._solve_subproblem(g, y, norms, i)

                # Update primal and dual coefficients.
                self.coef_ += (delta * X[i][:, np.newaxis]).T
                self.dual_coef_[:, i] += delta

            if it == 0:
                violation_init = violation_sum

            vratio = violation_sum / violation_init

            if self.verbose >= 1:
                print("iter", it + 1, "violation", vratio)

            if vratio < self.tol:
                if self.verbose >= 1:
                    print("Converged")
                break

        return self

    def predict(self, X):
        decision = np.dot(X, self.coef_.T)
        pred = decision.argmax(axis=1)
        return self._label_encoder.inverse_transform(pred)


'''if __name__ == '__main__':
    from sklearn.datasets import load_iris

    iris = load_iris()
    X, y = iris.data, iris.target

    clf = MulticlassSVM(C=0.1, tol=0.01, max_iter=100, random_state=0, verbose=1)
    clf.fit(X, y)
    print(clf.score(X, y))
    '''

In [ ]:
classifier = MulticlassSVM(C=0.1, tol=0.01, max_iter=100, random_state=0, verbose=1)
classifier.fit(x_train, y_train)
print(classifier.score(x_test, y_test))

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
y_pred1=classifier.predict(x_test)
cm = confusion_matrix(y_test,y_pred1)
accuracy= accuracy_score(y_test,y_pred1)

In [ ]:
cm

In [ ]:
accuracy

##### **SVM from scratch**

link: https://github.com/python-engineer/MLfromscratch/blob/master/mlfromscratch/svm.py


In [ ]:
class SVM:

    def __init__(self, learning_rate=0.001, lambda_param=0.01, n_iters=1000):
        self.lr = learning_rate
        self.lambda_param = lambda_param
        self.n_iters = n_iters
        self.w = None
        self.b = None


    def fit(self, X, y):
        n_samples, n_features = X.shape
        
        y_ = np.where(y <= 0, -1, 1)
        
        self.w = np.zeros(n_features)
        self.b = 0

        for _ in range(self.n_iters):
            for idx, x_i in enumerate(X):
                condition = y_[idx] * (np.dot(x_i, self.w) - self.b) >= 1
                if condition:
                    self.w -= self.lr * (2 * self.lambda_param * self.w)
                else:
                    self.w -= self.lr * (2 * self.lambda_param * self.w - np.dot(x_i, y_[idx]))
                    self.b -= self.lr * y_[idx]


    def predict(self, X):
        approx = np.dot(X, self.w) - self.b
        return np.sign(approx)

In [ ]:
model3 = SVM(n_iters=200)

In [ ]:
model3.fit(x_train, y_train)

In [ ]:
#print("score: ", model3.score(x_test, y_test))
y_pred1 = model3.predict(x_test)
cm = confusion_matrix(y_test,y_pred1)
accuracy= accuracy_score(y_test,y_pred1)

In [ ]:
print(cm)
print("accuracy: ", accuracy)

#### **SVM from andrew ng**

link: https://github.com/kenextra/SVMAlgorithm/blob/master/SVM.ipynb

In [ ]:
def linear_kernel(x1, x2):
    return np.dot(x1, x2)
    
def polynomial_kernel(x, y, p=3):
    return (1 + np.dot(x, y)) ** p

def gaussian_kernel(x, y, sigma=5.0):
    numerator = np.linalg.norm(x-y)**2
    denominator = 2 * (sigma ** 2)
    return np.exp(-numerator / denominator)

In [ ]:
class SVM(object):

    def __init__(self, kernel=linear_kernel, tol=1e-3, C=0.1,
                 max_passes=5, sigma=0.1):

        self.kernel = kernel
        self.tol = tol
        self.C = C
        self.max_passes = max_passes
        self.sigma = sigma
        self.model = dict()

    def __repr__(self):
        return (f"{self.__class__.__name__}("
                f"kernel={self.kernel.__name__}, "
                f"tol={self.tol}, "
                f"C={self.C}, "
                f"max_passes={self.max_passes}, "
                f"sigma={self.sigma}"
                ")")

    def svmTrain(self, X, Y):
        # Data parameters
        m = X.shape[0]

        # Map 0 to -1
        Y = np.where(Y == 0, -1, 1)

        # Variables
        alphas = np.zeros((m, 1), dtype=float)
        b = 0.0
        E = np.zeros((m, 1), dtype=float)
        passes = 0

        # Pre-compute the kernel matrix
        if self.kernel.__name__ == 'linear_kernel':
            print(f'Pre-computing {self.kernel.__name__} kernel matrix')
            K = X @ X.T

        elif self.kernel.__name__ == 'gaussian_kernel':
            print(f'Pre-computing {self.kernel.__name__} kernel matrix')
            X2 = np.sum(np.power(X, 2), axis=1).reshape(-1, 1)
            K = X2 + (X2.T - (2 * (X @ X.T)))
            K = np.power(self.kernel(1, 0, self.sigma), K)

        else:
            # Pre-compute the Kernel Matrix
            # The following can be slow due to lack of vectorization
            print(f'Pre-computing {self.kernel.__name__} kernel matrix')
            K = np.zeros((m, m))

            for i in range(m):
                for j in range(m):
                    x1 = np.transpose(X[i, :])
                    x2 = np.transpose(X[j, :])
                    K[i, j] = self.kernel(x1, x2)
                    K[i, j] = K[j, i]

        print('Training...')
        print('This may take 1 to 2 minutes')

        while passes < self.max_passes:
            num_changed_alphas = 0

            for i in range(m):

                E[i] = b + np.sum(alphas * Y * K[:, i].reshape(-1, 1)) - Y[i]

                if (Y[i] * E[i] < -self.tol and alphas[i] < self.C) or (Y[i] * E[i] > self.tol and alphas[i] > 0):
                    j = np.random.randint(0, m)
                    while j == i:
                        # make sure i is not equal to j
                        j = np.random.randint(0, m)

                    E[j] = b + np.sum(alphas * Y *
                                      K[:, j].reshape(-1, 1)) - Y[j]

                    # Save old alphas
                    alpha_i_old = alphas[i, 0]
                    alpha_j_old = alphas[j, 0]

                    # Compute L and H by (10) or (11)
                    if Y[i] == Y[j]:
                        L = max(0, alphas[j] + alphas[i] - self.C)
                        H = min(self.C, alphas[j] + alphas[i])
                    else:
                        L = max(0, alphas[j] - alphas[i])
                        H = min(self.C, self.C + alphas[j] - alphas[i])
                    if L == H:
                        # continue to next i
                        continue

                    # compute eta by (14)
                    eta = 2 * K[i, j] - K[i, i] - K[j, j]
                    if eta >= 0:
                        # continue to next i
                        continue

                    # compute and clip new value for alpha j using (12) and (15)
                    alphas[j] = alphas[j] - (Y[j] * (E[i] - E[j])) / eta

                    # Clip
                    alphas[j] = min(H, alphas[j])
                    alphas[j] = max(L, alphas[j])

                    # Check if change in alpha is significant
                    if np.abs(alphas[j] - alpha_j_old) < self.tol:
                        # continue to the next i
                        # replace anyway
                        alphas[j] = alpha_j_old
                        continue

                    # Determine value for alpha i using (16)
                    alphas[i] = alphas[i] + Y[i] * \
                        Y[j] * (alpha_j_old - alphas[j])

                    # Compute b1 and b2 using (17) and (18) respectively.
                    b1 = b - E[i] - Y[i] * (alphas[i] - alpha_i_old) * \
                        K[i, j] - Y[j] * (alphas[j] - alpha_j_old) * K[i, j]

                    b2 = b - E[j] - Y[i] * (alphas[i] - alpha_i_old) * \
                        K[i, j] - Y[j] * (alphas[j] - alpha_j_old) * K[j, j]

                    # Compute b by (19).
                    if 0 < alphas[i] < self.C:
                        b = b1
                    elif 0 < alphas[j] < self.C:
                        b = b2
                    else:
                        b = (b1 + b2) / 2
                    num_changed_alphas = num_changed_alphas + 1

            if num_changed_alphas == 0:
                passes = passes + 1
            else:
                passes = 0

            print('.', end='', flush=True)

        print('\n DONE! ')

        # Save the model
        idx = alphas > 0
        self.model['X'] = X[idx.reshape(1, -1)[0], :]
        self.model['y'] = Y[idx.reshape(1, -1)[0]]
        self.model['kernelFunction'] = self.kernel
        self.model['b'] = b
        self.model['alphas'] = alphas[idx.reshape(1, -1)[0]]
        self.model['w'] = np.transpose(np.matmul(np.transpose(alphas * Y), X))
        # return model

    def svmPredict(self, X):
        if X.shape[1] == 1:
            X = np.transpose(X)

        # Dataset
        m = X.shape[0]
        p = np.zeros((m, 1))
        pred = np.zeros((m, 1))

        if self.model['kernelFunction'].__name__ == 'linear_kernel':
            p = X.dot(self.model['w']) + self.model['b']

        elif self.model['kernelFunction'].__name__ == 'gaussian_kernel':
            # Vectorized RBF Kernel
            # This is equivalent to computing the kernel
            # on every pair of examples
            X1 = np.sum(np.power(X, 2), axis=1).reshape(-1, 1)
            X2 = np.transpose(np.sum(np.power(self.model['X'], 2), axis=1))
            K = X1 + (X2.T - (2 * (X @ (self.model['X']).T)))
            K = np.power(self.model['kernelFunction'](1, 0, self.sigma), K)
            K = np.transpose(self.model['y']) * K
            K = np.transpose(self.model['alphas']) * K
            p = np.sum(K, axis=1)

        else:
            for i in range(m):
                prediction = 0
                for j in range(self.model['X'].shape[0]):
                    prediction = prediction + self.model['alphas'][j] \
                        * self.model['y'][j] * \
                        self.model['kernelFunction'](np.transpose(
                            X[i, :]), np.transpose(self.model['X'][j, :]))

                p[i] = prediction + self.model['b']

        # Convert predictions into 0 and 1
        pred[p >= 0] = 1
        return pred

    def predict(self, X):
        if X.shape[1] == 1:
            X = np.transpose(X)

        # Dataset
        m = X.shape[0]
        p = np.zeros((m, 1))
        pred = np.zeros((m, 1))

        if self.model['kernelFunction'].__name__ == 'linear_kernel':
            p = X.dot(self.model['w']) + self.model['b']

        elif self.model['kernelFunction'].__name__ == 'gaussian_kernel':
            # Vectorized RBF Kernel
            # This is equivalent to computing the kernel
            # on every pair of examples
            X1 = np.sum(np.power(X, 2), axis=1).reshape(-1, 1)
            X2 = np.transpose(np.sum(np.power(self.model['X'], 2), axis=1))
            K = X1 + (X2.T - (2 * (X @ (self.model['X']).T)))
            K = np.power(self.model['kernelFunction'](1, 0, self.sigma), K)
            K = np.transpose(self.model['y']) * K
            K = np.transpose(self.model['alphas']) * K
            p = np.sum(K, axis=1)

        else:
            for i in range(m):
                prediction = 0
                for j in range(self.model['X'].shape[0]):
                    prediction = prediction + self.model['alphas'][j] \
                        * self.model['y'][j] * \
                        self.model['kernelFunction'](np.transpose(
                            X[i, :]), np.transpose(self.model['X'][j, :]))

                p[i] = prediction + self.model['b']

        # Convert predictions into 0 and 1
        pred[p >= 0] = 1
        return pred
